In [1]:
'''
Copyright 2024 Michael Koefinger

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
'''

'\nCopyright 2024 Michael Koefinger\n\nLicensed under the Apache License, Version 2.0 (the "License");\nyou may not use this file except in compliance with the License.\nYou may obtain a copy of the License at\n\n    http://www.apache.org/licenses/LICENSE-2.0\n\nUnless required by applicable law or agreed to in writing, software\ndistributed under the License is distributed on an "AS IS" BASIS,\nWITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\nSee the License for the specific language governing permissions and\nlimitations under the License.\n'

In [2]:
'''
Author: Michael Koefinger
Date: 18.01.2024
Notebook to determine coefficients of a Delta-Sigma modulator using FIR feedback based on [1]

[1] S. Pavan, "Continuous-Time Delta-Sigma Modulator Design Using the Method of Moments," in IEEE Transactions on Circuits and Systems I: Regular Papers, vol. 61, no. 6, pp. 1629-1637, June 2014, doi: 10.1109/TCSI.2013.2290846.

'''

'\nAuthor: Michael Koefinger\nDate: 18.01.2024\nNotebook to determine coefficients of a Delta-Sigma modulator using FIR feedback based on [1]\n\n[1] S. Pavan, "Continuous-Time Delta-Sigma Modulator Design Using the Method of Moments," in IEEE Transactions on Circuits and Systems I: Regular Papers, vol. 61, no. 6, pp. 1629-1637, June 2014, doi: 10.1109/TCSI.2013.2290846.\n\n'

In [3]:
import sympy as sym
from sympy import exp, factorial, binomial
import numpy as np
from scipy.stats import moment

In [4]:
''' Calculate the l-th moment of a (DAC) pulse based on its Laplace transform P(s)
    
    P: Laplace transform of pulse
    s: Laplace variable
    l: l-th moment of the pulse
    
'''
def moment_lap(P,s,l):
    deriv = P
    for i in range(l):
        deriv = deriv.diff(s)
    res = sym.limit(deriv, s, 0)
    res = res*(-1)**l
    return res
    

In [5]:
# Test moment_lap()
s = sym.Symbol('s')
n_fir = sym.symbols('n_fir')
tdelay = sym.symbols('tdelay')
Pdelay = exp(-tdelay*s)
P_nrz = (1-exp(-s))/s
P_rz = 2*(1-exp(-s/2))/s
P_nrz_fir = 1/n_fir*(1-exp(-n_fir*s))/s # lim s->0 ("0"/"0", L'Hopital) = n_fir/n_fir = 1
P_nrz_fir_neg1 = 1/(n_fir-1)*(1-exp(-(n_fir-1)*s))/s
P_triang_fir = P_nrz_fir*P_nrz_fir;
# P_triang_fir = P_nrz_fir*P_nrz_fir_neg1;

for i in range(4):
    print(moment_lap(P_nrz,s,i))

for i in range(4):
    print(moment_lap(P_nrz_fir,s,i))

for i in range(4):
    print(moment_lap(P_triang_fir,s,i))

1
1/2
1/3
1/4
1
n_fir/2
n_fir**2/3
n_fir**3/4
1
n_fir
7*n_fir**2/6
3*n_fir**3/2


In [6]:
''' Calculate the output of a chain of k intergrators given the Laplace transform of the input (DAC) pulse P(s)
    
    k: Number of integrators
    t: time variable
    P: Laplace transform of pulse
    s: Laplace variable
    
'''
def integ_chain_output(k, t, P, s):
    x_out = 0
    for l in range(k):
        x_out = x_out + ((-1)**l/(factorial(k-1))*binomial(k-1,l)*moment_lap(P,s,l)*t**(k-l-1))
    return x_out

In [7]:
# Test integ_chain_output
t = sym.symbols('t')
tdelay = sym.symbols('tdelay')
for i in range(5):
    print(integ_chain_output(i,t,P_nrz*Pdelay,s))

0
1
t - tdelay - 1/2
t**2/2 + t*(-tdelay - 1/2) + tdelay**2/2 + tdelay/2 + 1/6
t**3/6 + t**2*(-tdelay/2 - 1/4) + t*(tdelay**2/2 + tdelay/2 + 1/6) - tdelay**3/6 - tdelay**2/4 - tdelay/6 - 1/24


In [8]:
''' Calculate sampled sequence at the output of the chain of integrators, i.e. the loop filter of a Delta-Sigma modulator.

    a_list: List of coefficient names
    t: time variable
    N: Order of the loop filter, i.e. number of integrators
    P: Laplace transform of pulse
    s: Laplace variable

'''
def sampled_loopfilter_output(a_list, t, N, P, s):
    y_out = 0
    for k in range(1,N+1):
        a_k = a_list[k-1]
        x_k = integ_chain_output(k,t,P,s)
        y_out = y_out + a_k*x_k
        #print(k); print(a_k); print(x_k)
    return y_out

In [9]:
# Use method of moments to find tuned loop filter coefficients for a FIR DAC
t = sym.symbols('t')
[a1, a2, a3, a4] = sym.symbols('a1 a2 a3 a4')
[a1_fir, a2_fir, a3_fir, a4_fir] = sym.symbols('a1_fir a2_fir a3_fir a4_fir')
a_list = [a1, a2, a3, a4]
a_list_fir = [a1_fir, a2_fir, a3_fir, a4_fir]
N = 4
out_samp_nrz = sampled_loopfilter_output(a_list, t, N, P_nrz,s)
out_samp_nrz_fir = sampled_loopfilter_output(a_list_fir, t, N, P_nrz_fir,s)
y_nrz = out_samp_nrz.as_poly(t)
y_nrz_fir = out_samp_nrz_fir.as_poly(t)
coeff_proto_list = y_nrz.all_coeffs()
coeff_fir_list = y_nrz_fir.all_coeffs()
eq_list = []
for i in range(N):
    eq_list.append(coeff_fir_list[i]-coeff_proto_list[i])
    
sol = sym.solve(eq_list, a_list_fir)
simplified_sol = sym.simplify(sol)

In [10]:
sym.factor(simplified_sol)

{a1_fir: (24*a1 + 12*a2*n_fir - 12*a2 + 2*a3*n_fir**2 - 6*a3*n_fir + 4*a3 - a4*n_fir**2 + 2*a4*n_fir - a4)/24, a2_fir: (12*a2 + 6*a3*n_fir - 6*a3 + a4*n_fir**2 - 3*a4*n_fir + 2*a4)/12, a3_fir: (2*a3 + a4*n_fir - a4)/2, a4_fir: a4}

In [11]:
coeff_proto_list

[a4/6, a3/2 - a4/4, a2 - a3/2 + a4/6, a1 - a2/2 + a3/6 - a4/24]

In [12]:
coeff_fir_list

[a4_fir/6,
 a3_fir/2 - a4_fir*n_fir/4,
 a2_fir - a3_fir*n_fir/2 + a4_fir*n_fir**2/6,
 a1_fir - a2_fir*n_fir/2 + a3_fir*n_fir**2/6 - a4_fir*n_fir**3/24]

In [13]:
# Use method of moments to find tuned loop filter coefficients for a FIR DAC with unequal filter coeff.
t = sym.symbols('t')
[a1, a2, a3, a4] = sym.symbols('a1 a2 a3 a4')
[a1_fir, a2_fir, a3_fir, a4_fir] = sym.symbols('a1_fir a2_fir a3_fir a4_fir')
a_list = [a1, a2, a3, a4]
a_list_fir = [a1_fir, a2_fir, a3_fir, a4_fir]
N = 4
out_samp_nrz = sampled_loopfilter_output(a_list, t, N, P_nrz,s)
out_samp_triang_fir = sampled_loopfilter_output(a_list_fir, t, N, P_triang_fir,s)
y_nrz = out_samp_nrz.as_poly(t)
y_triang_fir = out_samp_triang_fir.as_poly(t)
coeff_proto_list = y_nrz.all_coeffs()
coeff_fir_list = y_triang_fir.all_coeffs()
eq_list = []
for i in range(N):
    eq_list.append(coeff_fir_list[i]-coeff_proto_list[i])
    
sol = sym.solve(eq_list, a_list_fir)
simplified_sol = sym.simplify(sol)

In [14]:
sym.factor(simplified_sol)

{a1_fir: (24*a1 + 24*a2*n_fir - 12*a2 + 10*a3*n_fir**2 - 12*a3*n_fir + 4*a3 + 2*a4*n_fir**3 - 5*a4*n_fir**2 + 4*a4*n_fir - a4)/24, a2_fir: (12*a2 + 12*a3*n_fir - 6*a3 + 5*a4*n_fir**2 - 6*a4*n_fir + 2*a4)/12, a3_fir: (2*a3 + 2*a4*n_fir - a4)/2, a4_fir: a4}

In [15]:
''' Calculate the output of a chain of k intergrators depending of the (k-1) moments of the DAC pulse shape
    
    k: Number of integrators
    t: time variable
    P: Laplace transform of pulse
    s: Laplace variable
    
'''
def integ_chain_output_general(k, t, u_k):
    x_out = 0
    for l in range(k):
        x_out = x_out + ((-1)**l/(factorial(k-1))*binomial(k-1,l)*u_k[l]*t**(k-l-1))
    return x_out

In [16]:
# Test integ_chain_output_general, same results as in [1]

k = 4;
[u0, u1, u2, u3] = sym.symbols('u0 u1 u2 u3') # lth order moments
u_k = np.array((u0, u1, u2, u3))
t = sym.symbols('t')
for i in range(1,k):
    print(integ_chain_output_general(i, t, u_k))


u0
t*u0 - u1
t**2*u0/2 - t*u1 + u2/2


In [17]:
''' Calculate sampled sequence at the output of the chain of integrators, i.e. the loop filter of a Delta-Sigma modulator.

    a_list: List of coefficient names
    t: time variable
    N: Order of the loop filter, i.e. number of integrators
    u_list: List of lth order moments names

'''
def sampled_loopfilter_output_general(a_list, t, N, u_list):
    y_out = 0
    for k in range(1,N+1):
        a_k = a_list[k-1]
        x_k = integ_chain_output_general(k,t, u_list)
        y_out = y_out + a_k*x_k
        #print(k); print(a_k); print(x_k)
    return y_out

In [18]:
# Test sampled_loopfilter_output_general, same results as in [1]

order = 3;
[a1, a2, a3, a4] = sym.symbols('a1 a2 a3 a4')
[u0, u1, u2, u3] = sym.symbols('u0 u1 u2 u3') # lth order moments
a_list = np.array((a1, a2, a3, a4))
u_list = np.array((u0, u1, u2, u3))
t = sym.symbols('t')
sym.factor(sampled_loopfilter_output_general(a_list, t, order, u_list).as_poly(t))

Poly(a3*u0/2*t**2 + (a2*u0 - a3*u1)*t + a1*u0 - a2*u1 + a3*u2/2, t, domain='QQ[u0,u1,u2,a1,a2,a3]')

In [19]:
# General Form of loop filter coefficients for CT-CT transformation
order = 4;
subsMom0_1 = True # assume that zero order moment, i.e. the area of the pulses is 1
t = sym.symbols('t')
[a1, a2, a3, a4] = sym.symbols('a1 a2 a3 a4')
[u0, u1, u2, u3] = sym.symbols('mu_0 mu_1 mu_2 mu_3') # lth order moments
[a1_, a2_, a3_, a4_] = sym.symbols('a1_t a2_t a3_t a4_t')
[u0_, u1_, u2_, u3_] = sym.symbols('mu0_t mu1_t mu2_t mu3_t') # lth order moments
a_list = np.array((a1, a2, a3, a4))
u_list = np.array((u0, u1, u2, u3))
a_list_ = np.array((a1_, a2_, a3_, a4_))
u_list_ = np.array((u0_, u1_, u2_, u3_))

y = sampled_loopfilter_output_general(a_list, t, order, u_list).as_poly(t)
y_ = sampled_loopfilter_output_general(a_list_, t, order, u_list_).as_poly(t)

# print(y)
# print(y_)

coeff = y.all_coeffs()
coeff_ = y_.all_coeffs()

eq_list = []
for i in range(order):
    eq_list.append(coeff_[i]-coeff[i])
    
sol = sym.solve(eq_list, a_list_)

if subsMom0_1:
    sol = sym.simplify(sol).subs({u0:1, u0_:1})

sol_list = []
sol_dict = {}
for i in range(order):
    lhs = a_list_[i]
    rhs = sol[lhs]
    equ = sym.Eq(lhs, sym.collect(rhs,a_list))
    sol_list.append(equ)
    sol_dict[lhs] = equ.rhs

In [20]:
sym.simplify(sol_dict) # won't pretty print w/o simplify

{a1_t: a1 + a2*(mu1_t - mu_1) + a3*(mu1_t**2 - mu1_t*mu_1 - mu2_t/2 + mu_2/2) + a4*(mu1_t**3 - mu1_t**2*mu_1 - mu1_t*mu2_t + mu1_t*mu_2/2 + mu2_t*mu_1/2 + mu3_t/6 - mu_3/6), a2_t: a2 + a3*(mu1_t - mu_1) + a4*(mu1_t**2 - mu1_t*mu_1 - mu2_t/2 + mu_2/2), a3_t: a3 + a4*(mu1_t - mu_1), a4_t: a4}

In [21]:
# Test some pulse shapes and their central moments
# import control.matlab as m
import scipy.signal as s
ntaps= 10;
denom = np.zeros((1,ntaps))
denom[0,0] = 1;
h_nrz = np.squeeze(np.array((1)))
h_fir_rect = np.squeeze(np.ones((1,ntaps))/ntaps)
h_fir_triang = s.windows.triang(ntaps)
h_fir_triang = h_fir_triang/np.sum(h_fir_triang)
print(h_nrz)
print(h_fir_rect)
print(h_fir_triang)

for i in range(4):
    print(moment(h_nrz,i))

for i in range(4):
    print(moment(h_fir_rect,i))

for i in range(4):
    print(moment(h_fir_triang,i))

1
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
[0.02 0.06 0.1  0.14 0.18 0.18 0.14 0.1  0.06 0.02]
1.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0032000000000000015
3.252606517456513e-20


/home/koefinger/tools/miniconda3/envs/dsm/lib/python3.12/site-packages/scipy/_lib/_util.py:1023: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return fun(*args, **kwargs)
